<a href="https://colab.research.google.com/github/shua1zhang/pytorch-playground/blob/master/PyTorch_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test file for PyTorch 


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

torch.cuda.is_available()

True